## Columbia University

This script serves as a basic tutorial for extracting courses of interest from a university. This is by no means the only (or even best way) to go about this process—so if you come up with a process that works better, feel free to implement! If you're unfamiliar with any of the libraries, the comments below annotate reasoning behind each.

In [237]:
import sys
import pandas as pd
import numpy as np
import time
import re
import urllib.request #handles urls
from urllib.request import urlopen
import urllib.parse 
import linkGrabber #extracts urls
import json #encodes/decodes json 
import csv 
import requests #downloads a webpage to scrape
from bs4 import BeautifulSoup, NavigableString, Tag #beautifulsoup pulls data from HTML
import nltk #NLP tasks
from nltk import word_tokenize
from nltk.stem import PorterStemmer #removes word endings
stemmer = PorterStemmer()

The first thing we want to do is set up a function for standard preprocessing. It's also useful to list all of the URLs we'll need to send requests to before scraping. We want all courses within a 2 year *academic* calendar (as opposed to an annual calendar). 

In [238]:
#keyword preprocessing
def preprocess(keyword):
    keyword = keyword.lower() #lowercase
    keyword = word_tokenize(keyword) #tokenize
    for word in keyword:
        keyword = stemmer.stem(word) #stem 
    return (keyword)

#course catalog URLs - 2 academic years 
#only 2019 is available, Fall(3), Summer(2), Spring(1)
# urls array

urls = [{"term": 'Fall 2019', "url":"?site=Directory_of_Classes&instr=&days=&semes=20191&hour="},
        {"term": 'Summer 2019', "url":'?si?site=Directory_of_Classes&instr=&days=&semes=20192&hour='},
        {"term": 'Spring 2019', "url":'?site=Directory_of_Classes&instr=&days=&semes=20193&hour='}]

link = 'https://doc.search.columbia.edu/classes/'

Next, we'll want to import our keyword csv, split our keyword lists, and preprocess them. The way the csv is set up, we'll want to split the words that are indicated as technical (`T`) or normative (`N`) and that we've chosen to include (`Y`). You'll notice that preprocessing is useful for some of our words but not for others. Here, we've chosen to manually alter words that are not usefully preprocessed. In this case, it means replacing instances of words that are stemmed to end in i.

[regex is a bitch here]

In [239]:
#import keywords
keywords = pd.read_csv("../keywords.csv")
technical = keywords[(keywords['Technical/Normative']=='T') & (keywords['Include']=='Y')].Keyword
normative = keywords[(keywords['Technical/Normative']=='N') & (keywords['Include']=='Y')].Keyword
normative = [preprocess(i) for i in normative]
technical = [preprocess(i) for i in technical] 

#replace keywords of interest
normative = [w.replace('privaci', 'privac') for w in normative]
normative = [w.replace('democraci', 'democra') for w in normative]
normative = [w.replace('equiti', 'equit') for w in normative]
normative = [w.replace('histori', 'histor') for w in normative]
normative = [w.replace('justice', 'justic') for w in normative]
normative = [w.replace('liberti', 'libert') for w in normative]
normative = [w.replace('philosophi', 'philosoph') for w in normative]
normative = [w.replace('societi', 'societ') for w in normative]
normative = [w.replace('polici', 'polic') for w in normative]

technical = [w.replace('ai', '^ai') for w in technical]
technical = [w.replace('cs', '^cs') for w in technical]
technical = [w.replace('ict', '^ict') for w in technical]
technical = [w.replace('ml', '^ml') for w in technical]
technical = [w.replace('nlp', '^nlp') for w in technical]

print(normative)
print(technical)

['account', 'critic', 'democra', 'discrimin', 'equal', 'equit', 'ethic', 'fair', 'femin', 'gender', 'govern', 'histor', 'inequ', 'justic', 'law', 'legal', 'libert', 'moral', 'norm', 'philosoph', 'polit', 'power', 'privac', 'race', 'religi', 'respons', 'right', 'secur', 'social', 'societ', 'surveil', 'transpar', 'valu', 'polic']
['^ai', 'algorithm', 'analyt', 'intellig', 'automat', 'code', 'comput', '^cs', 'cyber', 'data', 'digit', '^ict', 'inform', 'intelligen', 'internet', 'machin', '^ml', 'process', '^nlp', 'platform', 'program', 'robot', 'softwar', 'system', 'technolog']


The process behind extracting relevant courses works in two steps:
1. First, we want to find and extract all courses that contain any instance of a normative keyword.
2. Then, we want search within these courses to see if it also contains a technical keyword.

We initialize a data frame with columns for all of the course items we want to extract. It probably makes the most sense to standardize these feature names across all university scripts so that they're easier to merge in the final compiled dataset for all universities. Our items of interest are:
* The course title: `title`
* The department and course number: `dept_num`
* The course description: `description`
* The number of credits for the course: `credits`
* The course instructor: `instructor`
* The link to the course syllabus (if applicable): `syllabus`
* The university the course is extracted from: `university`
* The term that the course is offered during (fall, spring, summer / year): `term`
* The keyword that triggered the extraction (this is for auditing purposes): `keyword`

In [240]:
#init dfs
# columbia = pd.DataFrame(columns=['title','dept_num','description','credits','instructor',
#                                 'syllabus','university','term','keyword'])
# columbia = pd.DataFrame(columns=['title','university','term','keyword'])
columbia_list = []

The loop below executes part 1 of our extraction. It's long and kind of messy (sorry), so feel free to play around with the structure if you'd like. The key tasks here are to extract our items of interest based on our search queries and append them to our data frame.

In [ ]:
#roster search for all urls
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains

for url in urls:
    print("term", url["term"])
    
    #loop through all normative words and extract relevant elements 
    for word in normative: 
        print('-------------------')
        url_keyword = link + word + url['url'] #NOTE:this structure will likely be different between rosters!
        driver = webdriver.Chrome()
        driver.get(url_keyword)
        time.sleep(4)

        #the number of reponses
        elements = driver.find_elements_by_xpath('//*[@id="gsa-search-results"]/li')
        results = len(elements)
        print("elements", len(elements))
        
        #scraping each results
        for x in range(0, results):
            columbia_dict = {}
                   
            title = driver.find_element_by_xpath('//*[@id="gsa-search-results"]/li[' + str(x+1) +']/div/h3/a').text
            section = driver.find_element_by_xpath('//*[@id="gsa-search-results"]/li[' + str(x+1) +']/div/h3/a/span').text
            title = title.replace(section, '').strip()
            columbia_dict['title'] = title
            
            print(x, "/", len(elements))
#             print("SECTION", section)
            
            course_link = driver.find_element_by_xpath('//*[@id="gsa-search-results"]/li[' + str(x+1) +']/div/div[2]').text
#             print('course_link', course_link)
            
            #course_link.click()
#             driver.get(course_link)
#             time.sleep(3)
            
#             try:
#                 dept_nums = driver.find_element_by_xpath('//*[@id="col-right"]/table/tbody/tr[10]/td[2]').text
# #             print('dept_nums', dept_nums)
#             except:
#                 pass
        
#             descs = ''
            
#             try:
#                 credit = driver.find_element_by_xpath('//*[@id="col-right"]/table/tbody/tr[4]/td[2]').text
#             except:
#                 pass
            
#             try:
#                 profs = driver.find_element_by_xpath('//*[@id="col-right"]/table/tbody/tr[7]/td[2]').text
#             except:
#                 pass
            
#             syllabi = ''  
            columbia_dict['university'] = 'columbia university'
            columbia_dict['term'] = url["term"]  
            columbia_dict['keyword'] = word
            
#             columbia = columbia.append([title, dept_nums, descs, credit, profs, syllabi, uni, term, keyword])
            
            # only added if the title contains the word
            if word.upper() in title:
                columbia_list.append(columbia_dict)
            
        driver.close()


term Fall 2019
-------------------
elements 54
0 / 54
1 / 54
2 / 54
3 / 54
4 / 54
5 / 54
6 / 54
7 / 54
8 / 54
9 / 54
10 / 54
11 / 54
12 / 54
13 / 54
14 / 54
15 / 54
16 / 54
17 / 54
18 / 54
19 / 54
20 / 54
21 / 54
22 / 54
23 / 54
24 / 54
25 / 54
26 / 54
27 / 54
28 / 54
29 / 54
30 / 54
31 / 54
32 / 54
33 / 54
34 / 54
35 / 54
36 / 54
37 / 54
38 / 54
39 / 54
40 / 54
41 / 54
42 / 54
43 / 54
44 / 54
45 / 54
46 / 54
47 / 54
48 / 54
49 / 54
50 / 54
51 / 54
52 / 54
53 / 54
-------------------
elements 162
0 / 162
1 / 162
2 / 162
3 / 162
4 / 162
5 / 162
6 / 162
7 / 162
8 / 162
9 / 162
10 / 162
11 / 162
12 / 162
13 / 162
14 / 162
15 / 162
16 / 162
17 / 162
18 / 162
19 / 162
20 / 162
21 / 162
22 / 162
23 / 162
24 / 162
25 / 162
26 / 162
27 / 162
28 / 162
29 / 162
30 / 162
31 / 162
32 / 162
33 / 162
34 / 162
35 / 162
36 / 162
37 / 162
38 / 162
39 / 162
40 / 162
41 / 162
42 / 162
43 / 162
44 / 162
45 / 162
46 / 162
47 / 162
48 / 162
49 / 162
50 / 162
51 / 162
52 / 162
53 / 162
54 / 162
55 / 162
56 /

-------------------
elements 302
0 / 302
1 / 302
2 / 302
3 / 302
4 / 302
5 / 302
6 / 302
7 / 302
8 / 302
9 / 302
10 / 302
11 / 302
12 / 302
13 / 302
14 / 302
15 / 302
16 / 302
17 / 302
18 / 302
19 / 302
20 / 302
21 / 302
22 / 302
23 / 302
24 / 302
25 / 302
26 / 302
27 / 302
28 / 302
29 / 302
30 / 302
31 / 302
32 / 302
33 / 302
34 / 302
35 / 302
36 / 302
37 / 302
38 / 302
39 / 302
40 / 302
41 / 302
42 / 302
43 / 302
44 / 302
45 / 302
46 / 302
47 / 302
48 / 302
49 / 302
50 / 302
51 / 302
52 / 302
53 / 302
54 / 302
55 / 302
56 / 302
57 / 302
58 / 302
59 / 302
60 / 302
61 / 302
62 / 302
63 / 302
64 / 302
65 / 302
66 / 302
67 / 302
68 / 302
69 / 302
70 / 302
71 / 302
72 / 302
73 / 302
74 / 302
75 / 302
76 / 302
77 / 302
78 / 302
79 / 302
80 / 302
81 / 302
82 / 302
83 / 302
84 / 302
85 / 302
86 / 302
87 / 302
88 / 302
89 / 302
90 / 302
91 / 302
92 / 302
93 / 302
94 / 302
95 / 302
96 / 302
97 / 302
98 / 302
99 / 302
100 / 302
101 / 302
102 / 302
103 / 302
104 / 302
105 / 302
106 / 302
107 / 3

In [ ]:
columbia = pd.DataFrame(columbia_list)

#title only
# for word in normative:
#     columbia_df = columbia[columbia['title'].str.contains(word, flags = re.IGNORECASE)]

columbia

Now that we've extracted all courses containing a normative keyword of interest, we need to filter our courses to only return titles that contain a normative AND a technical keyword. This is the case for all words except instances of our preprocessed `privac` and `secur`, for which we want to return all courses, even if they don't contain two keywords. To do this, we'll split the courses into two data frames, apply our respective conditions, and then merge them back together. 

In [ ]:
exceptions = columbia.loc[(columbia['keyword']=='privac') | (columbia['keyword'] =='secur')]
exceptions

In [ ]:
#loop through technical keyword list, extract relevant titles
for word in technical:
    df = columbia[columbia['title'].str.contains(word, flags = re.IGNORECASE)]
    df['keyword2'] = word
    
#join keyword cols
df["keyword"] = df["keyword"].map(str) + "," + df["keyword2"]
df = df.drop(columns="keyword2")

df

NOTE: the above cell is likely not the best nor most simple way to execute this step! Feel free to take special liberties here. It's probably wise to pick out a few titles that you know should be returned manually, then check to see if the script is working as desired. 

In [ ]:
#combine dfs 
columbia = pd.concat([df, exceptions])
columbia

Lastly, we want to export our csv. Ideally, all csv files should be written to the courses directory in our repository. 

In [ ]:
#export as csv
columbia.to_csv('../courses/columbia.csv')